In [1]:


from pyspark import SparkConf, SparkContext
#sparkSession 드라이버 프로세스 얻기
conf = SparkConf().setMaster("local").setAppName("category-review-average")
sc = SparkContext.getOrCreate(conf=conf)

directory = "C:\\Users\\krlee\\JupyterLab\\fastcam_spark_and_flink"
filename = "restaurant_reviews.csv"

lines = sc.textFile(f"file:///{directory}/{filename}")
lines.collect()

Py4JError: org.apache.spark.api.python.PythonUtils.isEncryptionEnabled does not exist in the JVM

In [16]:
header = lines.first()

In [18]:
filtered_lines = lines.filter(lambda row: row != header)

In [19]:
filtered_lines.collect()

['0,짜장면,중식,125',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32',
 '3,떡볶이,분식,534',
 '4,라멘,일식,223   ',
 '5,돈가스,일식,52',
 '6,우동,일식,12',
 '7,쌀국수,아시안,312',
 '8,햄버거,패스트푸드,12',
 '9,치킨,패스트푸드,23']

In [20]:
def parse(row):
    fields = row.split(",")
    category = fields[2]
    reviews = int(fields[3])
    return (category, reviews)


In [21]:
categoryReviews = filtered_lines.map(parse)

In [22]:
categoryReviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [25]:
categoryReviewsCount = categoryReviews.mapValues(lambda x: (x, 1))
categoryReviewsCount.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [28]:
reduced = categoryReviewsCount.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [29]:
averages = reduced.mapValues(lambda x: x[0] / x[1])
averages.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]